# Simplification du réseau routier

In [3]:
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import split, unary_union, snap
from rtree import index 

import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint
from shapely.ops import split, unary_union, snap
from rtree import index as rtree_index  # Utilisation de rtree pour l'index spatial

import warnings
warnings.filterwarnings('ignore')

In [4]:
roads = gpd.read_file('C:/Users/hazim/Desktop/Nos_Cartes/data/reseau/provence-alpes-cote-d-azur-osm-geofabrik/gis_osm_roads_free_marseille_no_tunnel_no_motoway.gpkg')

In [5]:
print(roads.geometry.type.unique())

['MultiLineString']


## Préparation et extraction des LineString

In [6]:
def validate_geometry(geom):
    try:
        if geom is None or geom.is_empty or not geom.is_valid:
            return None
        return geom
    except Exception as e:
        print(f"Erreur lors de la validation de la géométrie : {e}")
        return None

# Supposons que roads soit déjà chargé sous forme de GeoDataFrame
roads['geometry'] = roads['geometry'].apply(validate_geometry)
roads = roads.dropna(subset=['geometry'])
print(f"Nombre de géométries valides : {len(roads)}")

roads = roads[roads['tunnel'] != 'T'] 
print(f"Lignes après exclusion des tunnels : {len(roads)}")

# Extraction de tous les segments individuels (LineString)
lines = []      # Liste des géométries de type LineString
line_ids = []   # Référence (ici osm_id) à la ligne d'origine

for idx, row in roads.iterrows():
    geom = row.geometry
    if geom.geom_type == 'LineString':
        lines.append(geom)
        line_ids.append(row.osm_id)
    elif geom.geom_type == 'MultiLineString':
        for sub_geom in geom.geoms:
            lines.append(sub_geom)
            line_ids.append(row.osm_id)

print(f"Nombre total de segments individuels extraits : {len(lines)}")

Nombre de géométries valides : 97953
Lignes après exclusion des tunnels : 96488
Nombre total de segments individuels extraits : 96517


In [7]:
print(roads.geometry.type.unique())

['MultiLineString']


## Calcul des points d'intersection entre segments

In [8]:
# Construction d'un index spatial pour accélérer la recherche
line_index = rtree_index.Index()
for pos, line in enumerate(lines):
    line_index.insert(pos, line.bounds)

intersection_points = []  # Liste pour stocker les intersections

# Pour chaque paire de lignes (en utilisant l'index), calculer l'intersection
for i, line in enumerate(lines):
    possible_indexes = list(line_index.intersection(line.bounds))
    for j in possible_indexes:
        if i >= j:
            continue  # éviter les redondances (i=j ou déjà traité)
        other_line = lines[j]
        inter = line.intersection(other_line)
        if inter.is_empty:
            continue
        # Si l'intersection est un Point
        if inter.geom_type == 'Point':
            intersection_points.append(inter)
        # Si l'intersection renvoie plusieurs points
        elif inter.geom_type == 'MultiPoint':
            intersection_points.extend(list(inter.geoms))
        # Si l'intersection est un LineString ou MultiLineString (cas de recouvrement)
        elif inter.geom_type in ['LineString', 'MultiLineString']:
            if inter.geom_type == 'LineString':
                intersection_points.append(Point(inter.coords[0]))
                intersection_points.append(Point(inter.coords[-1]))
            else:
                for sub in inter.geoms:
                    intersection_points.append(Point(sub.coords[0]))
                    intersection_points.append(Point(sub.coords[-1]))

print(f"Nombre de points d'intersection bruts : {len(intersection_points)}")

# Regrouper les points pour éliminer d'éventuels doublons
if intersection_points:
    intersection_union = unary_union(intersection_points)
    if intersection_union.geom_type == 'Point':
        intersection_union = MultiPoint([intersection_union])
else:
    intersection_union = None

Nombre de points d'intersection bruts : 160681


# Construction de l'index spatial pour les points d'intersection

In [9]:
# (pour limiter le nombre de points à considérer par ligne)
if intersection_union is not None:
    if intersection_union.geom_type == 'MultiPoint':
        pts = list(intersection_union.geoms)
    elif intersection_union.geom_type == 'Point':
        pts = [intersection_union]
    else:
        pts = []
else:
    pts = []

pt_index = rtree_index.Index()
for i, pt in enumerate(pts):
    pt_index.insert(i, pt.bounds)

def get_local_intersections(line, buffer=0.001):
    """
    Retourne l'union (MultiPoint) des points d'intersection se trouvant à
    moins de 'buffer' de la ligne.
    """
    minx, miny, maxx, maxy = line.bounds
    # Élargir légèrement l'enveloppe
    bbox = (minx - buffer, miny - buffer, maxx + buffer, maxy + buffer)
    candidate_ids = list(pt_index.intersection(bbox))
    selected_points = [pts[i] for i in candidate_ids if line.distance(pts[i]) < buffer]
    if selected_points:
        union_local = unary_union(selected_points)
        if union_local.geom_type == 'Point':
            union_local = MultiPoint([union_local])
        return union_local
    else:
        return None

## Découpage des lignes aux intersections (nœudisation)

In [10]:
tolerance = 1e-7  # Tolérance pour le snapping

# ----- Option 1 : Boucle séquentielle -----
final_segments = []  # Liste des segments finaux (LineString)
final_osm_ids = []   # Référence d'origine

for osm_id, line in zip(line_ids, lines):
    local_intersections = get_local_intersections(line, buffer=0.001)
    if local_intersections is not None:
        # Effectuer un snapping de la ligne sur les points locaux
        snapped_line = snap(line, local_intersections, tolerance)
        try:
            split_result = split(snapped_line, local_intersections)
        except Exception as e:
            print(f"Erreur lors du split de la ligne (osm_id={osm_id}) : {e}")
            continue
        for segment in split_result.geoms:
            if segment.length > 0 and segment.geom_type == 'LineString':
                final_segments.append(segment)
                final_osm_ids.append(osm_id)
    else:
        final_segments.append(line)
        final_osm_ids.append(osm_id)

print(f"Nombre total de segments après découpage (séquentiel) : {len(final_segments)}")

Nombre total de segments après découpage (séquentiel) : 194380


## Création du GeoDataFrame final contenant uniquement des LineString

In [11]:
final_network_gdf = gpd.GeoDataFrame({'osm_id': final_osm_ids},
                                     geometry=final_segments,
                                     crs=roads.crs)

print(f"Réseau final (uniquement des LineString) : {len(final_network_gdf)}")

Réseau final (uniquement des LineString) : 194380


## Simplifier les données pour sauvegarde

In [12]:
geo_path = "C:/Users/hazim/Desktop/Nos_Cartes/data/reseau/provence-alpes-cote-d-azur-osm-geofabrik/simplified_gis_osm_roads_free_marseille_no_tunnel_no_motoway_cleaned.gpkg"
final_network_gdf = final_network_gdf[['geometry']]
print(final_network_gdf.head())
final_network_gdf = final_network_gdf.to_crs("EPSG:2154")
final_network_gdf.to_file(geo_path, driver='GPKG', layer='edges')
print(f"Fichier sauvegardé avec succès : {geo_path}")

                                            geometry
0  LINESTRING (5.63579 43.29489, 5.63641 43.29524...
1  LINESTRING (5.57743 43.22003, 5.57762 43.22006...
2  LINESTRING (5.57659 43.22055, 5.57643 43.22051...
3  LINESTRING (5.59457 43.32271, 5.59464 43.32291...
4    LINESTRING (5.59486 43.32332, 5.59490 43.32339)
Fichier sauvegardé avec succès : C:/Users/hazim/Desktop/Nos_Cartes/data/reseau/provence-alpes-cote-d-azur-osm-geofabrik/simplified_gis_osm_roads_free_marseille_no_tunnel_no_motoway_cleaned.gpkg
